# A simple way to query SDSS's database from Python

### Import SDSS' sqlcl packages to query the SDSS database using web requests.

In [ ]:
import sqlcl # (DV) I modified sqlcl.py's code to query the DR13 and be usable in this jupyter notebook. 
             # It is included in this repository.
             # The original file is available at http://skyserver.sdss.org/dr2/en/help/Download/sqlcl/default.asp

### Import other useful packages to plot and manipulate data, and set plotting parameters with rc

In [ ]:
from __future__ import division

import sys
import wget

import numpy as np
from pandas import read_csv
from matplotlib import pyplot as plt
from matplotlib import rc

from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u

# Set font size for labels and axes ticks (http://matplotlib.org/users/customizing.html)
rc('font', size=18)
rc('axes', titlesize=15)
rc('axes', labelsize=14)

rc('xtick', labelsize=11)
rc('ytick', labelsize=11)

rc('lines', markersize=4)

rc('legend',fontsize=12)

%matplotlib inline
# Uncomment the following line if you have a mac with retina display
%config InlineBackend.figure_format='retina'

## Example 1: Print the result of a simple query

In [ ]:
sqlcl.query("select top 2 ra, dec from galaxy")

In [ ]:
print sqlcl.query("select top 2 ra, dec from galaxy").read()

## Exercise 2.1:  Plot all our quasars on sky using astropy

In [ ]:
query = """
SELECT TOP 1000 sp.ra, sp.dec, sp.z, f.ra as ra2, f.dec as dec2
FROM SpecPhoto AS sp 
INNER JOIN FIRST AS f 
    ON sp.objid = f.objid 
WHERE class = 'QSO'
ORDER BY sp.z DESC
        """
#ORDER BY sp.z DESC

results = read_csv(sqlcl.query(query), skiprows=1)

In [ ]:
results

In [ ]:
# See http://www.astropy.org/astropy-tutorials/plot-catalog.html for more details about plotting with astropy

# RA and DEC are in J2000 (from SpecObj view). 
ra = coord.Angle(results['ra']*u.degree)
ra = ra.wrap_at(180*u.degree)
dec = coord.Angle(results['dec']*u.degree)

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="mollweide")

# We change the labels -- default is degrees.
ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])

ax.set_xlabel('RA')
ax.set_ylabel('DEC')

scat = ax.scatter(ra.radian, dec.radian)
ax.grid(linestyle='--', linewidth=0.5)

## Exercise 2.2: Plot quasars as a function of redshift

In [ ]:
ra = coord.Angle(results['ra']*u.degree)
ra = ra.wrap_at(180*u.degree)
dec = coord.Angle(results['dec']*u.degree)
z = results['z']

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="mollweide")
scat = ax.scatter(ra.radian, dec.radian, c=z)#, cmap='coolwarm')
ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
ax.set_xlabel('RA')
ax.set_ylabel('DEC')

ax.grid(True, linewidth=0.5)

plt.colorbar(scat, ax=ax, label='z')

## Exercise 3: Evaluate the spatial localisation difference between SDSS and FIRST

In [ ]:
sub_results = results.loc[(results['ra2'] != 0.) & (results['dec2'] != 0.)]

plt.scatter(x=sub_results['ra']-sub_results['ra2'], 
            y=sub_results['dec']-sub_results['dec2'],
            c=sub_results['z'])
            #cmap='winter')
plt.xlabel('ra-ra2')
plt.ylabel('dec-dec2')

plt.title('Spatial localisation difference \n between SDSS and FIRST as a function of redshift')
plt.xlim([np.min(sub_results['ra']-sub_results['ra2']), np.max(sub_results['ra']-sub_results['ra2'])])
plt.ylim([np.min(sub_results['dec']-sub_results['dec2']), np.max(sub_results['dec']-sub_results['dec2'])])
plt.xticks(rotation=45)

plt.colorbar(label='z')

In [ ]:
query = """
SELECT TOP 1000 sp.ra, sp.dec, sp.z, sp.subClass
FROM SpecPhoto AS sp 
WHERE class = 'QSO'
        """
#ORDER BY sp.z DESC

results = read_csv(sqlcl.query(query), skiprows=1)

In [ ]:
results['subClass'].unique()

## Exercise 4.1: Download a spectrum based on information from query

### From the "Optical Spectra Per-Object Files" section (http://www.sdss.org/dr12/data_access/bulk/)

Spectra can be downloaded in bulk by filling a file (e.g. *speclist.txt*) with the list of spectra to be downloaded, 
with the following structure. 

    PLATE/spec-PLATE-MJD-FIBER.fits
    
e.g. 

    3586/spec-3586-55181-0016.fits
    3609/spec-3609-55201-0646.fits
    3661/spec-3661-55614-0020.fits
    ...
    
And the download can be done using `wget`. 

```wget --spider -nv -r -nH --cut-dirs=7 \
      -i speclist.txt \
      -B http://data.sdss3.org/sas/dr12/boss/spectro/redux/v5_7_0/spectra/```

In [ ]:
query = """
SELECT TOP 1 sp.ra, sp.dec, sp.z, sp.plate, sp.mjd, sp.fiberID, sp.survey, sp.boss_target1
FROM SpecPhoto AS sp 
WHERE class = 'QSO'
        """
#ORDER BY sp.z DESC

results = read_csv(sqlcl.query(query), skiprows=1)

In [ ]:
results

In [ ]:
for index, row in results.iterrows():
    print (row['plate'], row['mjd'], row['fiberID'])

In [ ]:
#http://data.sdss3.org/sas/dr13/boss/spectro/redux/v5_7_0/spectra/
core_path = 'https://data.sdss.org/sas/dr13/sdss/spectro/redux/26/spectra/'

for index, row in results.iterrows():
    plate = str(int(row['plate']))
    mjd = str(int(row['mjd']))
    fiberID = str(int(row['fiberID']))

    # Construct the file path
    file_path = plate + '/spec-' + plate + '-' + mjd + '-0' + fiberID + '.fits'
    
    url = core_path + file_path    
    print (url)
    filename = wget.download(url)

In [ ]:
filename

In [ ]:
spectrum = fits.open('spec-1725-54266-0252.fits')

spectrum[0].header

In [ ]:
spectrum[1].header

In [ ]:
for chunk in spectrum[1].data[0:10]:
    print (chunk)

In [ ]:
plt.plot(spectrum[1].data)

plt.ylabel('Flux')
plt.xlabel('Bin index')

# We need to use the information from the header to set the ticks correctly

### Exercise 4.2: Download and plot the data via the astroML package (fetch_sdss_spectrum)

In [ ]:
from astroML.datasets import fetch_sdss_spectrum

for index, row in results.iterrows():
    plate = int(row['plate'])
    mjd = int(row['mjd'])
    fiberID = int(row['fiberID'])

    spec = fetch_sdss_spectrum(int(plate), int(mjd), int(fiberID))

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3.75))
ax.plot(spec.wavelength(), spec.spectrum, '-k', lw=1)

# ax.set_xlim(3000, 10000)
# ax.set_ylim(25, 300)

ax.set_xlabel(r'$\lambda {(\rm \AA)}$')
ax.set_ylabel('Flux')
ax.set_title('Plate = %(plate)i, MJD = %(mjd)i, Fiber = %(fiberID)i' % locals())

plt.show()